In [1]:
import requests

from arcgis.gis import GIS
from arcgis.features import manage_data

import numpy as np
import pandas as pd

import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter("ignore", InsecureRequestWarning)

In [2]:
gis = GIS("home")

In [4]:
dd_id = "d37c3c2a6f1c4586baad82828bfc3c59"

dd_item = gis.content.get(dd_id)

geometries_layer = dd_item.layers[1]

# Item ID 0 is the output layer displayed in the dashboard
# COMMENTING OUT WHILE TESTING
#dashboard_layer = dd_item.layers[0]

test_id = "edb716da51bc4f7882d13d425ad08fd2"

test_item = gis.content.get(test_id)

dashboard_layer = test_item.layers[0]

In [5]:
api_url = r"https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fyDeclared ge 2013&$orderby=declarationDate desc"

response = requests.get(api_url)

data = response.json()

summaries_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])

summaries_df.shape

(1000, 28)

In [6]:
date_columns = ["declarationDate", "incidentBeginDate", "incidentEndDate", "disasterCloseoutDate", "lastIAFilingDate", "lastRefresh"]

for dc in date_columns:
    summaries_df[dc] = pd.to_datetime(summaries_df[dc], errors="coerce")

In [7]:
summaries_df["fipsFullCode"] = summaries_df["fipsStateCode"] + summaries_df["fipsCountyCode"]
summaries_df["fipsTribalCode"] = summaries_df["fipsStateCode"] + summaries_df["placeCode"]

summaries_df["COVID19"] = np.where(summaries_df["declarationTitle"].str.contains("COVID-19"), "Show only COVID-19", "Show only non-COVID-19")

In [8]:
entity_conditions = [
    summaries_df["designatedArea"] == "Statewide",
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] == "000"),
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] != "000")
]

entity_values = ["State or Equivalent", "Tribal Area or Equivalent", "County or Equivalent"]

summaries_df["Entity"] = np.select(entity_conditions, entity_values)

In [9]:
dashboard_sdf = dashboard_layer.query().sdf

dashboard_sdf.shape

(101, 24)

Okay sooooooo finally think I figured out the NEXT link in the Chain Of Things That Blew Up.
It's not picking up the tribal I need to add. But the line below DOES assume that the femaDecString is unique when it's NOT. If the Dec string is already in the dashboard for a county, it's not going to pick up that it needs to add it for tribal, DUH. I guess I will need a multi-field key to check for adds. DUH.

In [25]:
multi_key = ["femaDeclarationString", "Entity"]

pre_adds = summaries_df.merge(dashboard_sdf[multi_key], on=multi_key, how="left", indicator=True)

adds_df = pre_adds[pre_adds["_merge"] == "left_only"].drop(columns=["_merge"])

adds_df.shape

(2, 32)

In [26]:
state_field = "State_FIPS"
county_field = "Full_FIPS"
tribal_field1 = "AIANNHFP1"
tribal_field2 = "AIANNHFP2"
tribal_field3 = "AIANNHFP3"

key_fields_dict = {
    "State_FIPS": "fipsStateCode",
    "Full_FIPS": "fipsFullCode",
    "AIANNHFP1": "fipsTribalCode",
    "AIANNHFP2": "fipsTribalCode", # No idea why there are three of these...or why the Summaries sometimes use 2 and 3...
    "AIANNHFP3": "fipsTribalCode" # What a pain in the @$$...
}

### Get geometries associated with each Dec string for all three entity types
Since this has to be done technically 5 times (there are three different keys for Tribal), a function it is
(Yeah we'll throw everything in a function later, probably, since it's just tidy, but this requires it)

In [27]:
geometries_sdf = geometries_layer.query().sdf

In [28]:
adds_df.columns

Index(['femaDeclarationString', 'disasterNumber', 'state', 'declarationType',
       'declarationDate', 'fyDeclared', 'incidentType', 'declarationTitle',
       'ihProgramDeclared', 'iaProgramDeclared', 'paProgramDeclared',
       'hmProgramDeclared', 'incidentBeginDate', 'incidentEndDate',
       'disasterCloseoutDate', 'tribalRequest', 'fipsStateCode',
       'fipsCountyCode', 'placeCode', 'designatedArea',
       'declarationRequestNumber', 'lastIAFilingDate', 'incidentId', 'region',
       'designatedIncidentTypes', 'lastRefresh', 'hash', 'id', 'fipsFullCode',
       'fipsTribalCode', 'COVID19', 'Entity'],
      dtype='object')

In [29]:
state = "State or Equivalent"
county = "County or Equivalent"
tribal = "Tribal Area or Equivalent" 

def get_geometries(entity, field):

    # Checking Dec-string-per-entity
    entity_adds_df = adds_df[adds_df["Entity"] == entity]

    # NEED to specify an empty string for at least one of the suffixes here;
    # Otherwise I have no "Entity" field and my Dashboard layer and adds feature collection won't have matching fields
    merged = entity_adds_df.merge(geometries_sdf, left_on=[key_fields_dict[field]], right_on=field, suffixes=('', '_2'))

    print(merged.shape)

    return merged

state_geometries = get_geometries(state, state_field)

county_geometries = get_geometries(county, county_field)

tribal1_geometries = get_geometries(tribal, tribal_field1)

tribal2_geometries = get_geometries(tribal, tribal_field2)

tribal3_geometries = get_geometries(tribal, tribal_field3)

(0, 51)
(0, 51)
(2, 51)
(0, 51)
(0, 51)


### Stack the entity dataframes
Once I've generated rows for all Entity / Dec string combinations, I can just stack them for simple processing below.

In [30]:
adds_geometries = pd.concat([state_geometries, county_geometries, tribal1_geometries, tribal2_geometries, tribal3_geometries])

### Testing 9/7
* Current problem: I deleted the Tribal entities in the testing data (4 rows); the script isn't picking up anything new to add.
* Must be that I am not calculating my tribal keys correctly; looking into this now.

In [31]:
if adds_geometries.empty:
    
    print("Found no new Declarations to add! We're done here.")

else:

    adds_feature_collection = adds_geometries.spatial.to_feature_collection()

    remove_fields = ["OBJECTID", "Shape__Area", "Shape__Length", "SHAPE"]

    dissolve_fields = [f["name"] for f in dashboard_layer.properties.fields if not f["name"] in remove_fields]

    print(dissolve_fields)

    adds_dissolved = manage_data.dissolve_boundaries(adds_feature_collection, dissolve_fields=dissolve_fields)

    adds_features = adds_dissolved.query().features

    message = dashboard_layer.edit_features(adds_features)

    message

['Entity', 'femaDeclarationString', 'disasterNumber', 'state', 'declarationType', 'declarationDate', 'fyDeclared', 'incidentType', 'declarationTitle', 'incidentBeginDate', 'incidentEndDate', 'disasterCloseoutDate', 'tribalRequest', 'fipsStateCode', 'declarationRequestNumber', 'lastIAFilingDate', 'incidentId', 'region', 'designatedIncidentTypes', 'COVID19']


{"cost": 0.002}
